In [1]:
%matplotlib widget

In [2]:
import os
import pickle
import copy

import pandas as pd
import seaborn as sns
import numpy
import torch
import scipy
import scipy.stats

import pyro
import pyro.infer
import pyro.infer.mcmc
import pyro.distributions as dist
import torch.distributions.constraints as constraints
from tqdm.auto import tqdm

import matplotlib.pyplot as plot
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D
import warnings
warnings.filterwarnings('ignore')
sns.set(style="whitegrid")
import glob
import matplotlib.pyplot as plt

from torch.distributions.normal import Normal
from torch.distributions.log_normal import LogNormal

from multi_virt_v2 import *

## Load Params

In [3]:

base_dir='/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt_sync'
file_name=f'lr-0.0001-steps-$5500-alpha-lognormal-identity-dim*_theta-normal-identity_nosubsample_1.00_0.40_particles8/params.p'
exp_dir=os.path.join(base_dir, file_name)


print(exp_dir)

/Users/phumon/Documents/Research/nlu-test-sets/params_mvirt_sync/lr-0.0001-steps-$5500-alpha-lognormal-identity-dim*_theta-normal-identity_nosubsample_1.00_0.40_particles8/params.p


In [4]:
def get_posterior(exp_dir, dimension):
    posterior_stats = {}
    
    pyro.clear_param_store()
    pyro.get_param_store().load(exp_dir)
    with torch.no_grad():
        pyro_param_dict = dict(pyro.get_param_store().named_parameters())
    
    for k, v in pyro_param_dict.items():
        if k == "g mu":
            n_items = v.size(0)
        elif k == "t mu":
            n_models = v.size(0)
        posterior_stats[k] = v.mean().item()
    
    betas = Normal(posterior_stats['b mu']*torch.ones(n_items, dimension), torch.exp(torch.tensor(posterior_stats['b logstd'])))
    log_gamma = Normal(posterior_stats['g mu']*torch.ones(n_items), torch.exp(torch.tensor(posterior_stats['g logstd'])))
    alphas = LogNormal(posterior_stats['a mu'] * torch.ones(n_items, dimension), torch.exp(torch.tensor(posterior_stats['a logstd'])))
    thetas = Normal(posterior_stats['t mu'] * torch.ones(n_models, dimension), torch.exp(torch.tensor(posterior_stats['t logstd'])))
    probs = {
                "beta": betas,
                "log_gamma": log_gamma,
                "alpha": alphas,
                "theta": thetas
    }
    
    return posterior_stats, probs, n_items, n_models

In [5]:
def get_prior(item_param_std, alpha_std, dimension, n_items, n_models):
    # Generate params
    betas = Normal(torch.zeros(n_items, dimension), torch.tensor(item_param_std))
    log_gamma = Normal(torch.zeros(n_items), torch.tensor(item_param_std))

    alphas = LogNormal(0 * torch.ones(n_items, dimension), torch.tensor(alpha_std))

    # Generate thetas
    thetas = Normal(0 * torch.ones(n_models, dimension), torch.tensor(item_param_std))
    probs = {
                "beta": betas,
                "log_gamma": log_gamma,
                "alpha": alphas,
                "theta": thetas
    }
    
    return probs

In [6]:
def get_marginal_loglik(prior_probs, posterior_probs, dimension, n_particles=10):
    log_weights = []
    for i in range(n_particles):
        #log p(x) = logsumexp(log p(x|z_k) + log p(z_k) - log q(z_k), dim=DIM) - log(K)
        log_weight = 0
        prior_values = {}
        for k, dist_param in prior_probs.items():
            prior_tmp = prior_probs[k].sample()
            prior_values[k] = prior_tmp
            posterior_tmp = posterior_probs[k].sample()
            log_weight += (prior_probs[k].log_prob(prior_tmp) - posterior_probs[k].log_prob(posterior_tmp)).sum()
        
        gamma = sigmoid(prior_values['log_gamma'])
        alphas = prior_values['alpha']
        thetas = prior_values['theta']
        betas = prior_values['beta']
        if dimension > 1:
            prob = gamma[None, :] + (1.0 - gamma[None, :]) * sigmoid(torch.sum(alphas[None, :, :] * (thetas[:, None] - betas[None, :]).squeeze(), dim=-1))
        else:
            betas=betas.squeeze()
            gamma=gamma.squeeze()
            alphas=alphas.squeeze()
            thetas=thetas.squeeze()
            prob = gamma[None, :] + (1.0 - gamma[None, :]) * sigmoid(alphas[None, :] * (thetas[:, None] - betas[None, :]))
        lik_dist = torch.distributions.bernoulli.Bernoulli(prob)
        lik_values = lik_dist.sample()
        
        
        log_weight += lik_dist.log_prob(lik_values).sum()
        log_weights.append(log_weight.item())
    print(log_weights)
    marginal = torch.logsumexp(torch.tensor(log_weights), 0) - torch.log(torch.tensor(n_particles))
    return marginal
    #return lik, prob

In [7]:
marginal_logprob_list=[]
correct_list=[]
dim_list=[]
item_param_std=1; alpha_std=0.4

for filepath in glob.iglob(exp_dir):
    dimension = int(filepath.split('/')[-2].split('_')[0].rsplit('-',1)[1][3:])
    dim_list.append(dimension)
    
    posterior_stats, posterior_probs, n_items, n_models = get_posterior(filepath, dimension)
    prior_probs = get_prior(item_param_std, alpha_std, dimension, n_items, n_models)
    marginal = get_marginal_loglik(prior_probs, posterior_probs, dimension)
    
    marginal_logprob_list.append(marginal.item())
    
    
    
    

[-15991.0166015625, -16066.56640625, -16634.5546875, -15519.5908203125, -18762.345703125, -15306.13671875, -18005.1328125, -19186.91796875, -16879.60546875, -16800.58984375]
[-17975.08203125, -16520.216796875, -16124.12890625, -18429.7265625, -18251.59765625, -17730.220703125, -17190.837890625, -17095.607421875, -16362.701171875, -16486.322265625]
[-15421.1728515625, -14920.5224609375, -17155.546875, -16431.419921875, -16884.353515625, -17090.876953125, -13768.7275390625, -16899.806640625, -11728.8037109375, -14935.193359375]
[-16810.115234375, -18892.05859375, -16427.779296875, -15869.2353515625, -16938.8984375, -17039.359375, -14880.25, -15419.775390625, -15154.0205078125, -16090.96875]
[-16550.1328125, -18577.87109375, -16685.1328125, -15252.1494140625, -15334.419921875, -15568.2001953125, -16942.66015625, -17039.7265625, -18487.55078125, -16938.38671875]
[-15709.982421875, -15500.9638671875, -16291.2080078125, -17359.583984375, -16128.974609375, -15467.611328125, -14958.109375, -16

In [8]:
plot_data = list(zip(dim_list, marginal_logprob_list))
plot_data.sort(key=lambda x: x[0])
dim, marginals = list(zip(*plot_data))

plot_data

[(1, -17337.2109375),
 (2, -16126.431640625),
 (3, -15254.4521484375),
 (4, -15308.439453125),
 (5, -14882.552734375),
 (6, -14960.412109375),
 (7, -13384.125),
 (8, -11731.1064453125)]

In [9]:
plot_data = list(zip(dim_list, marginal_logprob_list))
plot_data.sort(key=lambda x: x[0])
dim, marginals = list(zip(*plot_data))

plot_data

[(1, -17337.2109375),
 (2, -16126.431640625),
 (3, -15254.4521484375),
 (4, -15308.439453125),
 (5, -14882.552734375),
 (6, -14960.412109375),
 (7, -13384.125),
 (8, -11731.1064453125)]

In [10]:

plt.figure()
plt.plot(dim, marginals)
plt.xlabel('Dim')
plt.ylabel('log p(x)') 

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:

plt.figure()
plt.plot(dim, acc)
plt.xlabel('Dim')
plt.ylabel('reconstruction acc') 

plt.show()